# TP d'optimisation sous incertitude - ENAC - M2RO - 2021 - 2ème partie


## Avant de commencer...


L'implémentation se fait toujours en utilisant le langage de programmation <tt>python</tt> dans ce <tt>notebook</tt> et la bibliothèque <tt>pulp</tt> (https://pypi.org/project/PuLP/). Pour chaque question, un  squelette vous est fourni et doit être complété. 

<b>Il est attendu un compte rendu avec les réponses à toutes les questions (5 pages max) et la remise des codes via le notebook.</b>


La cellule ci-dessous charge les librairies et effectue un test du bon fonctionnement de puLp. Si tout se passe bien, vous devez obtenir la sortie suivante:

<tt> === Programme de test (ENAC) ===<br>
Résolution d'un PLNE basique... Ok: x1 = 0.0, x2 = 2.0.<br>
=== Fin du programme === </tt>

In [2]:
# -*- encoding: utf-8 -*-

from __future__ import print_function

from airland import read_instance, read_scenarios
import pulp
from pulp import LpStatus

import argparse
import os
import sys
import verifier

import test_pulp
test_pulp.test_pulp()

=== Programme de test (ENAC) ===
Résolution d'un PLNE basique... Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mtds/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/874d0b9aed694e7182b3d04f65554033-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/874d0b9aed694e7182b3d04f65554033-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 29 RHS
At line 37 BOUNDS
At line 38 ENDATA
Problem MODEL has 7 rows, 2 columns and 14 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-7) rows, 0 (-2) columns and 0 (-14) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value -2
After Postsolve, objective -2, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -2 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU sec

/home/mtds/anaconda3/lib/python3.11/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Le processus d'atterrissage d'un avion dans un aéroport

La figure ci-dessous illustre les principes de contrôle de la navigation d'un avion commercial sur un exemple de vol Orly-Toulouse. Pendant le vol de croisière le contrôle est assuré par un centre de contrôle régional, par exemple Paris et Bordeaux sur les deux avant derniers tronçons. Ensuite la tour de contrôle de l'aéroport prend le relai pour le contrôle d'approche.

Un avion $i$ entrant dans la zone de contrôle de l'aéroport à l'instant $a_i$ a une date d'atterrissage au plus tôt $e_i$ correspondant à la vitesse maximale qu'il peut atteindre et une date d'atterrissage au plus tard $d_i$ correspondant à la date d'atterrissage maximale possible compte-tenu du carburant disponible et de la possibilité de tourner en rond dans la zone d'attente. Par ailleurs, compte-tenu de sa vitesse de croisière, un avion a également une date d'atterrissage préférentielle $t_i$. En cas de violation de cette date préférentielle, des coûts unitaires en cas de retard  $g_i$ et en cas d'avance $h_i$ sont à payer. De plus, entre l'atterrissage de deux avions $i$ et $j$, un temps de séparation minimum noté $s_{ij}$ dépendant du type des deux appareils doit être respecté.

Le rôle des contrôleurs aériens est d'attribuer à tout avion une date d'atterrissage effective $T_i\in[e_i,d_i]$ ainsi qu'une piste d'atterrissage (lorsque plusieurs sont disponibles). Il n'est pas toujours possible de choisir $T_i=t_i$ pour tout avion en raison de possibilités d'engorgement suite à des arrivées proches de différents avions.

![title](images/procedure-atterrissage.png)
(Source: Cinq sur cinq numéro 15 - septembre 2016)


## Le problème statique déterministe (Correction de la question 1)

Le problème statique détermine consiste, à partir d'un ensemble d'avion donné $P$, dont toutes les caractéristiques sont supposées connues à l'avance, et d'un ensemble de piste d'atterrissages $R$, à trouver les dates d'atterrissage qui respectent les contraintes décrites dans la section précédente et qui minimisent la somme pondérée des coûts d'avance et de retard. Il s'agit d'un problème d'ordonnancement NP-difficile, étudié dans l'article de Beasley *et al.* [1].  Un ensemble d'instances du problème proposé par les auteurs de cet article est téléchargeable sur le site de la OR-library (ces instances sont égalements fournies dans l'archive du TP), une bibliothèque de problèmes classiques de Recherche Opérationnelle (http://people.brunel.ac.uk/%7Emastjjb/jeb/orlib/airlandinfo.html).

<i> Exemple : Dans le cours 1, les données de l'instance "airland1.txt" à 10 avions et un exemple de solution  de coût total 1210 pour une seule piste d'atterrissage sont décrits.</i>

Dans cet article, le problème est formulé comme un programme linéaire en nombre entiers (PLNE). Une version simplifiée <b> et incomplète </b> est proposée ci-dessous.

En plus des paramètres déjà présentés, on introduit les variables de décisions binaires $x_{ij}$ qui indiquent que l'avion $i\in P$ atterrit avant l'avion $j\in P$ sur la même piste d'atterrissage et les variables binaires $y_{ir}$ qui indiquent que l'avion $i\in P$ atterrit sur la piste $r\in R$. On définit également des variables de décision continues : $T_i$ pour la date d'atterrissage de l'avion $i\in P$, $L_i$ pour le retard de l'avion $i\in P$, $E_i$ pour l'avance de l'avion $i\in P$ et $z$ pour l'objectif.

\begin{align}
  \min \quad & z\label{obj} & &\quad  (1) \\ % \sum_{i\in P} ( g_i L_i + h_u E_i ) \\
  \mbox{s.c. } \quad & T_i \geq e_i & \forall i\in P &\quad (2)\\
             &T_i \leq d_i & \forall i\in P&\quad  (3) \\
              &x_{ij}+x_{ji}  \leq 3 - y_{ir} - y_{jr} & \forall i,j\in P,i<j,\forall r\in R&\quad(4)\\
              &x_{ij}+x_{ji}  \geq y_{ir} + y_{jr} - 1 & \forall i,j\in P,i<j,\forall r\in R&\quad(5) \\
              & \sum_{r\in R} y_{ir} = 1 & \forall i\in P&\quad(6)\\
             &T_j \geq T_i + s_{ij} x_{ij} - M_{ij} (1 - x_{ij} ) & \forall i,j\in P,i\neq j &\quad(7)\\
             & L_i \geq T_i-t_i & \forall i\in P&\quad(8)\\
             & E_i \geq t_i - T_i &\forall i \in P &\quad(9)\\
             & x_{ij}\in\{0,1\} & \forall i,j\in P,i\neq j &\quad(10)\\
             & y_{ir}\in\{0,1\} & \forall i \in P, \forall r\in R &\quad(11)\\
             & L_i, E_i, T_i \geq 0 &  \forall i \in P &\quad(12)
\end{align}

Les contraintes (2),(3) indiquent qu'un avion ne peut atterrir avant sa date d'arrivée au plus tôt ou après sa date d'arrivée au plus tard. Les contraintes (5),(6)  définissent les retards et les avances. Les domaines de définition des variables de décisions sont donnés par (7),(8),(9).

[1]: J.E. Beasley, M. Krishnamoorthy, Y.M. Sharaiha and
D. Abramson. Scheduling aircraft landings - the static case, Transportation Science, vol.34, 2000, pp180-197 (https://core.ac.uk/download/pdf/337240.pdf).


Nous commençons par lire une instance du problème. Pour tester avec d'autres instances il suffit de mentionner un autre nom de fichier contenu dans le répertoire <tt>instance</tt> de l'archive fournie.

In [3]:
import io
instance_name = 'instances/airland3.txt'
instance = open(instance_name)

runways=2

P, F, A, E, L, T, S, G, H = read_instance(instance)
print('Nombre d\'avions P='+str(P))
print()
print('Nombre de pistes R='+str(P))
print()
print('Largeur de la fenêtre de fixation des dates d\'atterrissage F='+str(F))
print()
print('Dates d\'apparition des avions A='+str(A))
print()
print('Dates d\'atterrissage au plus tôt E='+str(E))
print()
print('Dates d\'atterrissage au plus tard L='+str(L))
print()
print('Dates d\'atterrissage préférentielles T='+str(T))
print()
print('Pénalités unitaires de retard G='+str(G))
print()
print('Pénalités unitaires d\'avance H='+str(H))
print()

P = range(P)
print('matrice des temps de séparation')
import numpy as np
print(np.array(S))
#for i in P:
#    print(S[i])

R = range(runways)


Nombre d'avions P=20

Nombre de pistes R=20

Largeur de la fenêtre de fixation des dates d'atterrissage F=10

Dates d'apparition des avions A=[0, 82, 59, 28, 126, 20, 110, 23, 42, 42, 57, 39, 186, 175, 139, 235, 194, 162, 69, 76]

Dates d'atterrissage au plus tôt E=[75, 157, 134, 103, 201, 95, 185, 98, 117, 117, 132, 114, 261, 250, 214, 310, 269, 237, 144, 151]

Dates d'atterrissage au plus tard L=[486, 628, 561, 565, 735, 524, 664, 523, 578, 569, 615, 551, 834, 790, 688, 967, 818, 726, 607, 624]

Dates d'atterrissage préférentielles T=[82, 197, 160, 117, 261, 106, 229, 108, 132, 130, 149, 126, 336, 316, 258, 409, 338, 287, 160, 169]

Pénalités unitaires de retard G=[30.0, 10.0, 10.0, 30.0, 10.0, 30.0, 10.0, 30.0, 30.0, 30.0, 30.0, 30.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 30.0, 30.0]

Pénalités unitaires d'avance H=[30.0, 10.0, 10.0, 30.0, 10.0, 30.0, 10.0, 30.0, 30.0, 30.0, 30.0, 30.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 30.0, 30.0]

matrice des temps de séparation
[[99999    15    1



Le PLNE ci-dessous résout le problème statique (correction de la question 1)


In [4]:
from pulp import lpDot, lpSum, \
     LpProblem, LpMinimize, \
     LpVariable, LpBinary, LpInteger


def get_problem_q1(**kargs):

    P = kargs['P']  # Liste d'avions (0, 1, 2, ..., N)
    R = kargs['R']  # Pistes d'atterrissage (0, 1, 2, ..., M)
    e = kargs['e']  # Dates d'atterrissage au plus tôt
    d = kargs['d']  # Dates d'atterrissage au plus tard
    t = kargs['t']  # Dates d'atterrissage préférentielles
    s = kargs['s']  # Délais de séparation
    g = kargs['g']  # Coûts unitaires de retard
    h = kargs['h']  # Coûts unitaires d'avance

    # -- Calcul des bigs M
    M = [[d[i] - e[j] for j in P] for i in P]

    # -- Création du problème
    problem = LpProblem("Ordonnancement des atterrissages - Le problème statique", LpMinimize)

    # -- Création des variables

    # T - Date d'atterrissage
    T = [LpVariable('T_{}'.format(i), e[i], d[i]) for i in P]

    # E - Avance de l'avion
    E = [LpVariable('E_{}'.format(i), 0) for i in P]
    # L - Retard de l'avion
    L = [LpVariable('L_{}'.format(i), 0) for i in P]

    # x - x_ij = 1 si l'avion i attérit avant l'avion j
    x = [[LpVariable('x_{}_{}'.format(i, j), cat=LpBinary) for j in P] for i in P]

    # y  - y_ir = 1 si l'avion i atterit sur la piste r
    y = [[LpVariable('y_{}_{}'.format(i, r), cat=LpBinary) for r in R] for i in P]

    # -- Création de l'objectif

    problem += lpDot(g, L) + lpDot(h, E)

    # -- Création des contraintes
    for i in P:
        for j in filter(lambda x: x > i, P):
            for r in R:
                problem += x[i][j] + x[j][i] >= y[i][r] + y[j][r] - 1

    for i in P:
        problem += lpSum(y[i]) == 1

    for i in P:
        for j in filter(lambda x: x != i, P):
            problem += T[j] >= T[i] + s[i][j] * x[i][j] - M[i][j] * (1 - x[i][j])

    for i in P:
        problem += L[i] >= T[i] - t[i]
        problem += E[i] >= t[i] - T[i]

    print(problem)
    return problem

Construction des arguments pour la fonction get_problem et exécution de la méthode <tt>solve</tt> pour le solveur <tt>CBC</tt> (<i>Coin-or branch and cut</i>) avec <tt>msg=True</tt> activant l'affichage des sorties du solveur (vous pouvez voir le nombre de variables et de contraintes puis les inforemations sur les étapes de résolution). Un temps de calcul maximum peut être défini (ici 60 secondes). Faites varier ce temps pour voir la différence dans les solutions trouvées sur les différentes instances.

In [5]:
kargs = {'P': P, 'R': R, 'e': E, 'd': L, 't': T, 's': S, 'g': G, 'h': H}
problem = get_problem_q1(**kargs)
status = problem.solve(pulp.PULP_CBC_CMD(msg=True,maxSeconds=60))

/home/mtds/anaconda3/lib/python3.11/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/home/mtds/anaconda3/lib/python3.11/site-packages/pulp/apis/coin_api.py:95: UserWarning: Parameter maxSeconds is being depreciated for timeLimit
  warnings.warn("Parameter maxSeconds is being depreciated for timeLimit")


Ordonnancement_des_atterrissages_-_Le_problème_statique:
MINIMIZE
30.0*E_0 + 10.0*E_1 + 30.0*E_10 + 30.0*E_11 + 10.0*E_12 + 10.0*E_13 + 10.0*E_14 + 10.0*E_15 + 10.0*E_16 + 10.0*E_17 + 30.0*E_18 + 30.0*E_19 + 10.0*E_2 + 30.0*E_3 + 10.0*E_4 + 30.0*E_5 + 10.0*E_6 + 30.0*E_7 + 30.0*E_8 + 30.0*E_9 + 30.0*L_0 + 10.0*L_1 + 30.0*L_10 + 30.0*L_11 + 10.0*L_12 + 10.0*L_13 + 10.0*L_14 + 10.0*L_15 + 10.0*L_16 + 10.0*L_17 + 30.0*L_18 + 30.0*L_19 + 10.0*L_2 + 30.0*L_3 + 10.0*L_4 + 30.0*L_5 + 10.0*L_6 + 30.0*L_7 + 30.0*L_8 + 30.0*L_9 + 0.0
SUBJECT TO
_C1: x_0_1 + x_1_0 - y_0_0 - y_1_0 >= -1

_C2: x_0_1 + x_1_0 - y_0_1 - y_1_1 >= -1

_C3: x_0_2 + x_2_0 - y_0_0 - y_2_0 >= -1

_C4: x_0_2 + x_2_0 - y_0_1 - y_2_1 >= -1

_C5: x_0_3 + x_3_0 - y_0_0 - y_3_0 >= -1

_C6: x_0_3 + x_3_0 - y_0_1 - y_3_1 >= -1

_C7: x_0_4 + x_4_0 - y_0_0 - y_4_0 >= -1

_C8: x_0_4 + x_4_0 - y_0_1 - y_4_1 >= -1

_C9: x_0_5 + x_5_0 - y_0_0 - y_5_0 >= -1

_C10: x_0_5 + x_5_0 - y_0_1 - y_5_1 >= -1

_C11: x_0_6 + x_6_0 - y_0_0 - y_6_0 >=

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mtds/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5200cb458ccc4508ba1ebd9e800182e6-pulp.mps sec 60 timeMode elapsed branch printingOptions all solution /tmp/5200cb458ccc4508ba1ebd9e800182e6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 825 COLUMNS
At line 4486 RHS
At line 5307 BOUNDS
At line 5768 ENDATA
Problem MODEL has 820 rows, 480 columns and 2780 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 60
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0004I processed model has 800 rows, 460 columns (400 integer (400 of which binary)) and 2761 elements
Cutoff increment increased from 1e-05 to 9.9999
C

Affichage de l'instance, du statut de résolution et de l'objectif obtenu :

In [6]:
print('Instance "{}"{}: {} planes'.format(
    instance_name.split(os.sep)[1][:-4], '', len(P)))
print('Status: ', LpStatus[problem.status])
# utiliser cela pour nouvelles versions pulp :
# print('Status: ', LpStatus[problem.getSolutionStatus()])

print('Objective: ', problem.objective.value())
print()


Instance "airland3": 20 planes
Status:  Optimal
Objective:  60.0



Récupération des valeurs des variables de décision et affichage de la solution :

In [7]:
vars = problem.variablesDict()
# dates de début
t = [round(vars['T_'+str(i)].value()) for i in P]
Q = None
#affectations aux pistes
y = []
y = []
for i in P:
    for r in R:
        if vars['y_{}_{}'.format(i, r)].roundedValue():
            y.append(r)
# Affichage

for i in P:
    print('  Plane {} appearing at {}, landing at t={} on runway {} '
          '({}, [{}, {}]).'.format(P[i], A[i], t[i], y[i], T[i],
                                   E[i], L[i]))


  Plane 0 appearing at 0, landing at t=82 on runway 0 (82, [75, 486]).
  Plane 1 appearing at 82, landing at t=197 on runway 0 (197, [157, 628]).
  Plane 2 appearing at 59, landing at t=160 on runway 0 (160, [134, 561]).
  Plane 3 appearing at 28, landing at t=117 on runway 0 (117, [103, 565]).
  Plane 4 appearing at 126, landing at t=261 on runway 0 (261, [201, 735]).
  Plane 5 appearing at 20, landing at t=106 on runway 0 (106, [95, 524]).
  Plane 6 appearing at 110, landing at t=229 on runway 1 (229, [185, 664]).
  Plane 7 appearing at 23, landing at t=108 on runway 1 (108, [98, 523]).
  Plane 8 appearing at 42, landing at t=132 on runway 1 (132, [117, 578]).
  Plane 9 appearing at 42, landing at t=130 on runway 0 (130, [117, 569]).
  Plane 10 appearing at 57, landing at t=149 on runway 1 (149, [132, 615]).
  Plane 11 appearing at 39, landing at t=124 on runway 1 (126, [114, 551]).
  Plane 12 appearing at 186, landing at t=336 on runway 1 (336, [261, 834]).
  Plane 13 appearing at 1

Il est possible de vérifier la faisabilité de la solution en appelant la fonction <tt>verifier.verify</tt>

In [8]:
p, w = verifier.verify(t, y, E, T, L, S, G, H)
print('Ok (obj. = {}).'.format(w) if p else w)

i=0 diff=0
i=1 diff=0
i=2 diff=0
i=3 diff=0
i=4 diff=0
i=5 diff=0
i=6 diff=0
i=7 diff=0
i=8 diff=0
i=9 diff=0
i=10 diff=0
i=11 diff=-2
i=12 diff=0
i=13 diff=0
i=14 diff=0
i=15 diff=0
i=16 diff=0
i=17 diff=0
i=18 diff=0
i=19 diff=0
Ok (obj. = 60.0).


## 4 - Le problème dynamique ou online

On considère maintenant un processus dynamique de planification. Les informations sur les avions ne sont pas toutes disponibles au début mais arrivent au cours du temps alors que d'autres avions sont déjà en train ou très proches d'atterrir. Plus précisément, à chaque avion $i$ est associée une date d'apparition $a_i\leq e_i$. Chaque date $a_i$ correspond à un instant où la planification doit être relancée pour incorporer le nouvel avion $i$ dans le planning.

Soit $P(t)$ les avions $i\in P$ tels que  $a_i\leq t$, c'est à dire l'ensemble des avions apparus avant la date $t$. Le problème dynamique consiste en $|P|$ étapes: À chaque étape $q$, un avion $\sigma(q)$ apparait à la date $t_q=a_{\sigma(q)}$ et le problème d'ordonnancement des avions $P(t_q)$ doit  être résolu en ignorant les avions qui apparaîtront ultérieurement. $T^q_i$ désigne ainsi la date d'atterrissage allouée par la planification à l'avion $i\in P(q)$ à l'étape $q$. A une étape, les avions dont l'atterrissage est déjà passé ou trop proche ne peuvent pas se voir attribuer une nouvelle date. On pose ainsi la contrainte $T^q_i=T^{q-1}_i$ pour tout avion appartenant à $P(t_q)\cap P(t_{q-1})$  tel que $T^{q-1}_i\leq t_q+\Delta$ où $\Delta\geq 0$ est un paramètre appelé la période de gel de la planification.

<i>Exemple : Dans le cours 1, les dates d'arrivées $a_i$ sont présentées pour l'instance {\tt airland1} et l'exemple du problème à résoudre à l'étape $t_q=60$ est donné.</i>


Adapter le PLNE statique pour incorporer les nouvelles contraintes.


In [11]:
from __future__ import print_function

from pulp import lpDot, lpSum, \
     LpProblem, LpMinimize, \
     LpVariable, LpBinary, LpInteger


def get_problem_q4(**kargs):

    P = kargs['P']  # Liste d'avions (0, 1, 2, ..., N)
    R = kargs['R']  # Pistes d'atterrissage (0, 1, 2, ..., M)
    e = kargs['e']  # Dates d'atterrissage au plus tôt
    d = kargs['d']  # Dates d'atterrissage au plus tard
    t = kargs['t']  # Dates d'atterrissage préférentielles
    s = kargs['s']  # Délais de séparation
    g = kargs['g']  # Coûts unitaires de retard
    h = kargs['h']  # Coûts unitaires d'avance
    q = kargs['q']  # Dates d'atterrissage trouvées par la planification précédente
    tq = kargs['tq']  # Date de la planification
    delta = kargs['delta']  # Delta

    # à compléter....
    # -- Calcul des bigs M à faire !
    M = [[d[i] - e[j]for j in P]for i in P]
    # -- Création du problème
    problem = LpProblem("Ordonnancement des atterrissages - Le problème statique", LpMinimize)

    # -- Création des variables

    # T - Date d'atterrissage
    T = [LpVariable('T_{}'.format(i), e[i], d[i]) for i in P]

    # E - Avance de l'avion
    E = [LpVariable('E_{}'.format(i), 0) for i in P]
    # L - Retard de l'avion
    L = [LpVariable('L_{}'.format(i), 0) for i in P]

    # x - x_ij = 1 si l'avion i attérit avant l'avion j
    x = [[LpVariable('x_{}_{}'.format(i, j), cat=LpBinary) for j in P] for i in P]

    # y  - y_ir = 1 si l'avion i atterit sur la piste r
    y = [[LpVariable('y_{}_{}'.format(i, r), cat=LpBinary) for r in R] for i in P]
    
    # -- Création de l'objectif
    
    problem += lpSum([g[i]*L[i] + h[i]*E[i] for i in P]) 
    
    # -- Création des contraintes

        
    for i in P:
        for j in filter(lambda x: x != i, P):
            problem += T[j] >= T[i] + s[i][j] * x[i][j] - M[i][j] * (1 - x[i][j])

    for i in P:
        problem += L[i] >= T[i] - t[i]
        problem += E[i] >= t[i] - T[i]


    for i in P:
        for j in P:
            for r in R:
                problem += x[i][j] + x[j][i] >= y[i][r] + y[j][r] - 1

            
    for i in P:
        problem += lpSum([y[i][r] for r in R]) == 1 
    
    
    return problem


Lancer le programme ci-dessous  qui insère les avions un par un dans le problème en suivant l'ordre de la date d'arrivée A[i], puis qui appelle le solver à chaque arrivée, et qui affiche ensuite la solution.

In [10]:
idx = sorted(range(len(A)), key=lambda i: A[i])
planes = idx

# Switch parameters in ascending order for regarding appearance time
A = [A[i] for i in idx]
E = [E[i] for i in idx]
L = [L[i] for i in idx]
T = [T[i] for i in idx]
G = [G[i] for i in idx]
H = [H[i] for i in idx]
S = [[S[i][j] for j in idx] for i in idx]

Tp = []  # Previous planes affectation

Delta = -1e10 #100

for p in P:
    tq = A[idx[p]]
    c = p + 1
    problem = get_problem_q4(
        P=range(c), R=R, e=E, d=L, t=T, g=G, h=H, s=S,
        q=Tp, tq=tq, delta=Delta)
    status = problem.solve(pulp.PULP_CBC_CMD(msg=False,maxSeconds=60))
    vars = problem.variablesDict()
    _Tp = [vars['T_'+str(i)].roundedValue() for i in range(c)]

    for i in range(len(Tp)):
        if Tp[i] <= tq + Delta and Tp[i] != _Tp[i]:
            print('Error: Trying to assign a new time plane {} after '
                  'limit.'.format(i))
            exit(1)
    Tp = _Tp


vars = problem.variablesDict()
# dates de début
t = [round(vars['T_'+str(i)].value()) for i in P]
Q = None
#affectations aux pistes
y = []
y = []
for i in P:
    for r in R:
        if vars['y_{}_{}'.format(i, r)].roundedValue():
            y.append(r)
# Affichage

print('Delta='+str(Delta))
for i in P:
    print('  Plane {} appearing at {}, landing at t={} on runway {} '
          '({}, [{}, {}]).'.format(P[i], A[i], t[i], y[i], T[i],
                                   E[i], L[i]))

# verification de la solution et affichage de l'objectif
p, w = verifier.verify(t, y, E, T, L, S, G, H)
print('Ok (obj. = {}).'.format(w) if p else w)

Delta=-10000000000.0
  Plane 0 appearing at 0, landing at t=82 on runway 0 (82, [75, 486]).
  Plane 1 appearing at 20, landing at t=106 on runway 1 (106, [95, 524]).
  Plane 2 appearing at 23, landing at t=108 on runway 0 (108, [98, 523]).
  Plane 3 appearing at 28, landing at t=117 on runway 1 (117, [103, 565]).
  Plane 4 appearing at 39, landing at t=126 on runway 0 (126, [114, 551]).
  Plane 5 appearing at 42, landing at t=134 on runway 0 (132, [117, 578]).
  Plane 6 appearing at 42, landing at t=130 on runway 1 (130, [117, 569]).
  Plane 7 appearing at 57, landing at t=149 on runway 0 (149, [132, 615]).
  Plane 8 appearing at 59, landing at t=160 on runway 1 (160, [134, 561]).
  Plane 9 appearing at 69, landing at t=160 on runway 0 (160, [144, 607]).
  Plane 10 appearing at 76, landing at t=169 on runway 0 (169, [151, 624]).
  Plane 11 appearing at 82, landing at t=197 on runway 1 (197, [157, 628]).
  Plane 12 appearing at 110, landing at t=229 on runway 0 (229, [185, 664]).
  Plan

Lancer sur différentes instances avec différentes nombre de pistes et différentes valeurs de Delta, réaliser des tableaux et comparer l'objectif avec le problème statique. Conclusion ? Que se passe -t-il si on prend un $\Delta$ négatif suffisamment petit ?
